In [1]:
import requests
import datetime
import os
import json
import re


asr_url = 'http://95.64.227.126:4514/asr' 
generate_url = "http://95.64.227.126:4502/api/generate" 

def strip_parasite_words(text):
    parasite_words= ['как бы', 'реально', 'типа', 'короче', 'на самом деле', 'практически', 'так сказать', 'вообще-то', 'кстати', 'ну', 'вот', 'блин', 'так вот', 'в принципе',
                 'значит', 'собственно говоря', 'в общем-то', 'в натуре', 'это самое', 'как говорится', 'прикинь', 'допустим', 'стало быть', 'ща', 'чё', 'реально', 'ну вот', 
                 'в самом деле', 'все такое', 'да ладно', 'допустим', 'итак', 'как его', 'как сказать', 'как-то так', 'конкретно', 'походу', 'практически', 'прикол', 'прямо говоря',
                 'мягко говоря', 'погоди', 'хорош', 'так', 'о', 'скажем', 'грубо говоря', 'честно говоря', 'таки', 'типа того', 'это же', 'чего', 'ладно', 'ну ок', 'ладушки',
                 'эээ', 'эммм', 'угу', 'хмм', 'ага', 'ммм', 'амм', 'вот', 'ну как', 'ну', 'ну это', 'эй']
    for word in parasite_words:
        text = re.sub(rf"\b{word}\b", '', text)
    return text
    
def mp4towav(mp4, timestamp):
    output_file = f'audio_{timestamp}.wav'
    os.system(f'ffmpeg -i {input_file} -vn -acodec pcm_s16le -ar 44100 -ac 2 -strict -2 {output_file}')
    return output_file

def truncsribe(audio_file, timestamp):
    audio_file_path = audio_file
    data = {
        'encode': True,  
        'task': 'transcribe',
        'language': 'ru',
        'output': 'txt',
    }
    #print(audio_file_path)
    with open(audio_file_path, 'rb') as audio_file:
        files = {'audio_file': audio_file}
        response = requests.post(asr_url, files=files, data=data)

    if response.status_code == 200:
        #print('Файл успешно загружен:', response.text)
        text = strip_parasite_words(response.text)
        output_file_path = f'text_{timestamp}.txt'
        #print(output_file_path)
        with open(output_file_path, 'w', encoding='utf-8') as output_file:
            output_file.write(text)
        print(f'Текст сохранен в файл: {output_file_path}')
        return output_file_path
    else:
       # print('Ошибка при загрузке файла:', response.status_code, response.text)
         print('Ошибка при загрузке файла:', response.status_code)
         return ""


def summarize(trunscribe_file, timestamp):
    api_url = "http://95.64.227.126:4502/api/generate"
    model_name = "qwen2.5:32b"
    #text_file_path = 'text_20250408_0859.txt'

    try:
        with open(trunscribe_file, 'r', encoding='utf-8') as file:
            text_to_summarize = file.read()
    except FileNotFoundError:
        print(f"Файл {trunscribe_file} не найден.")
        text_to_summarize = None

    if text_to_summarize:
        # Разделяем текст на части (например, по абзацам)
        payload = {
            "prompt": text_to_summarize,
            "model": model_name,
            "max_tokens": 100000
        }
        try:
            # Отправляем POST-запрос с параметром stream=True
            response = requests.post(api_url, json=payload, stream=True)

            # Собираем весь ответ из потока
            full_response = ""
            for line in response.iter_lines():  # Читаем каждую строку потока
                if line:  # Проверяем, что строка не пустая
                    data = line.decode("utf-8")  # Декодируем байты в строку
                    try:
                       json_data = json.loads(data)  # Преобразуем строку в словарь с помощью json.loads
                       if "response" in json_data:
                           full_response += json_data["response"]  # Добавляем часть ответа
                       if json_data.get("done", False):  # Если генерация завершена
                           break
                    except Exception as e:
                        print(f"Ошибка обработки: {e}")

            print("Ответ модели:", full_response)
            # timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
            #timestamp = '20250408_1608' 
            summary_output_path = f'summary_{timestamp}.txt'
            with open(summary_output_path, 'w', encoding='utf-8') as summary_file:
                summary_file.write(full_response)
            return summary_output_path
        except Exception as e:
            print(f"Ошибка обработки: {e}")
            return ""


def protokol(summarize_file, timestamp):
    api_url = "http://95.64.227.126:4502/api/generate"
    model_name = "qwen2.5:32b"
    #text_file_path = 'text_20250408_0859.txt'
    # input_text_file = f'summary_20250414_0735.txt'
  
    # Проверка существования файла
    if not os.path.exists(summarize_file):
        print(f"Файл с текстом {summarize_file} не найден.")
        exit(1)

    # Чтение текста
    with open(summarize_file, 'r', encoding='utf-8') as f:
        raw_text = f.read()

    # Подготовка prompt
    prompt = f"""В тексте: 
    Написать структурированный протокол совещания
    Протокол включает разделы:
    - Заголовок: Протокол.
    - Заголовок: Участники совещания (перечисли только имена и фамилии участников, которые упомянуты в тексте).
    - Заголовок: Повестка дня (выдели основные темы обсуждения).
    - Заголовок: Основные решения и выводы (укажи ключевые решения по каждой теме повестки).
    - Заголовок: План действий (опиши конкретные задачи, которые были назначены участникам, с указанием ответственных лиц и сроков, если они обсуждались).
    - Заголовок: Дополнительные замечания (включи важные комментарии или вопросы, которые не вошли в основные пункты).

    Текст:
    {raw_text}
    """

    payload = {
        "model": model_name,
        "prompt": prompt,
        "temperature": 0.1,
        #"max_tokens": 2048,
    }

    print("Отправляем текст на коррекцию в Qwen...")
    response = requests.post(generate_url, json=payload)

    if response.status_code == 200:
        lines = response.text.strip().split('\n')
        result_parts = []

        for line in lines:
            try:
                data = json.loads(line)
                result_parts.append(data.get("response", ""))
            except Exception as e:
                print(f"Ошибка при обработке строки:\n{line}\n{e}")

        corrected_text = ''.join(result_parts)

        corrected_file_path = f'text_protokol_{timestamp}.txt'
        with open(corrected_file_path, 'w', encoding='utf-8') as f:
            f.write(corrected_text)

        print(f'Исправленный текст сохранен в файл: {corrected_file_path}')
        return corrected_file_path
    else:
        print("Ошибка при обращении к Qwen:", response.status_code, response.text)
        return ""




#загрузка видео файла
input_file = 'record4.mp4'


if not os.path.exists(input_file):
    print(f"Файл {input_file} не найден.")
    exit(1)


timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M")
print('timestamp=', timestamp)

wav_file = mp4towav(input_file, timestamp)
if not os.path.exists(wav_file):
    print(f"Файл {wav_file} не найден.")
    exit(1)

trunscribe_file = truncsribe(wav_file, timestamp)
if trunscribe_file != "":
    summary_file = summarize(trunscribe_file, timestamp)
    if summary_file != "" :
        print( f"суммаризация в файле {summary_file}")
    else:
        print( "суммаризация не удалась")

protocol_file = protokol(summary_file, timestamp)
if protocol_file != "":
    protocol_file = summarize(trunscribe_file, timestamp)
    if summary_file != "" :
        print( f"суммаризация в файле {summary_file}")
    else:
        print( "суммаризация не удалась")




timestamp= 20250414_1111


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Текст сохранен в файл: text_20250414_1111.txt
Ответ модели: Похоже, вы ведете обсуждение по поводу подготовки презентации для какого-то мероприятия или события. Здесь несколько ключевых моментов:

1. **Рекрутер**: Вы собираетесь представить рекрутера и делать это через 4-5 слайдов. Вам нужно определиться, кому передать эти слайды (Кириллу и Марии), чтобы они встроили их в общую презентацию.

2. **Союз**: Нужно решить, нужны ли слайды Союза или достаточно просто упомянуть о нем.

3. **Документация и общение**: Вам необходимо связаться с Кириллом и Мария для согласования деталей презентации и переходов между разделами.

4. **Время работы**: Вы планируете завершить работу над презентацией до полтретьего (15:30), после чего соберетесь вместе, чтобы обсудить последние изменения и уточнения.

Кроме того, вы решили временно остановить запись. Вот несколько шагов для дальнейших действий:

### Шаги:
1. **Создание слайдов по рекрутеру**:
   - Соберите все необходимую информацию.
   - Создайте 4-